In [1]:
!pip install optuna
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.5 MB/s eta 0:00:00


In [2]:
import os
import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
from google.colab import drive
drive.mount('/content/drive')

# 데이터 로드
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이터/train_processed2.csv')
submit_test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이터/test_processed2.csv')

Mounted at /content/drive


# 데이터 분할

In [4]:
# Target 및 Train/Test Split
target_df = train_df['가격(백만원)']
train_df = train_df.drop(columns=['가격(백만원)'])
train_df, test_df, target_df, test_target_df = train_test_split(
    train_df, target_df, test_size=0.2, random_state=42
)

# 모델 함수 정의

In [5]:
# Optuna Objective 함수 정의
def objective_lightgbm(trial, train_df, target_df, test_df, test_target_df):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
    }
    model = LGBMRegressor(**params, random_state=42)
    model.fit(train_df, target_df)
    predictions = model.predict(test_df)
    return np.sqrt(mean_squared_error(test_target_df, predictions))

def objective_catboost(trial, train_df, target_df, test_df, test_target_df):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'depth': trial.suggest_int('depth', 4, 15),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
    }
    model = CatBoostRegressor(**params, verbose=False, random_state=42)
    model.fit(train_df, target_df)
    predictions = model.predict(test_df)
    return np.sqrt(mean_squared_error(test_target_df, predictions))

def objective_xgb(trial, train_df, target_df, test_df, test_target_df):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
    }
    model = xgb.XGBRegressor(objective='reg:squarederror', **params)
    model.fit(train_df, target_df)
    predictions = model.predict(test_df)
    return np.sqrt(mean_squared_error(test_target_df, predictions))

# 추가 모델 최적화를 위한 함수 (Bagging, RandomForest, ExtraTrees, GradientBoosting, DecisionTree)
def objective_sklearn(trial, model_class, train_df, target_df, test_df, test_target_df):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000) if 'n_estimators' in model_class().get_params() else None,
        'max_depth': trial.suggest_int('max_depth', 3, 30),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
    }
    params = {k: v for k, v in params.items() if v is not None}
    model = model_class(**params, random_state=42)
    model.fit(train_df, target_df)
    predictions = model.predict(test_df)
    return np.sqrt(mean_squared_error(test_target_df, predictions))

# Optuna 최적화 함수

In [6]:
# Optuna 최적화 함수
def optimize_model(model_name, objective_func, train_df, target_df, test_df, test_target_df):
    db_dir = "/content/optuna"
    os.makedirs(db_dir, exist_ok=True)
    storage_name = f"sqlite:///{db_dir}/optuna_{model_name}.db"

    try:
        study = optuna.create_study(
            storage=storage_name,
            study_name=model_name,
            direction='minimize',
            sampler=TPESampler(multivariate=True, seed=42),
        )
    except:
        study = optuna.load_study(study_name=model_name, storage=storage_name)

    study.optimize(
        lambda trial: objective_func(trial, train_df, target_df, test_df, test_target_df),
        n_trials=50,
    )
    return study

In [7]:
# 모델 최적화 및 예측
models = {
    'LightGBM': (objective_lightgbm, LGBMRegressor),
    'CatBoost': (objective_catboost, CatBoostRegressor),
    'XGBoost': (objective_xgb, xgb.XGBRegressor),
    'RandomForest': (objective_sklearn, RandomForestRegressor),
    'ExtraTrees': (objective_sklearn, ExtraTreesRegressor),
    'DecisionTree': (objective_sklearn, DecisionTreeRegressor),
    'GradientBoosting': (objective_sklearn, GradientBoostingRegressor),
}

In [ ]:
import functools

# 모델 최적화 및 예측
studies = {}
predictions = {}

print("=== 모델 최적화 시작 ===")
for model_name, (objective_func, model_class) in models.items():
    print(f"Optimizing {model_name}...")

    if model_name in ['Bagging', 'RandomForest', 'ExtraTrees', 'DecisionTree', 'GradientBoosting']:
        # objective_sklearn을 사용하는 경우
        study_objective = functools.partial(
            objective_sklearn,
            model_class=model_class,
            train_df=train_df,
            target_df=target_df,
            test_df=test_df,
            test_target_df=test_target_df,
        )
    else:
        # LightGBM, CatBoost, XGBoost 등 다른 모델
        study_objective = functools.partial(
            objective_func,
            train_df=train_df,
            target_df=target_df,
            test_df=test_df,
            test_target_df=test_target_df,
        )

    # 모델 최적화
    studies[model_name] = optuna.create_study(
        study_name=model_name,
        direction='minimize',
        sampler=TPESampler(multivariate=True, seed=42),
    )
    studies[model_name].optimize(study_objective, n_trials=50)

    # 최적화된 모델로 예측 수행
    best_model = model_class(**studies[model_name].best_params, random_state=42)
    best_model.fit(train_df, target_df)
    predictions[model_name] = best_model.predict(test_df)

# 단순 평균 앙상블
ensemble_mean = sum(predictions.values()) / len(predictions)
rmse_mean = np.sqrt(mean_squared_error(test_target_df, ensemble_mean))
r2_mean = r2_score(test_target_df, ensemble_mean)

# 가중치 기반 앙상블
weights = [1 / studies[model].best_value for model in predictions.keys()]
weights = np.array(weights) / sum(weights)
ensemble_weighted = sum(weights[i] * list(predictions.values())[i] for i in range(len(weights)))
rmse_weighted = np.sqrt(mean_squared_error(test_target_df, ensemble_weighted))
r2_weighted = r2_score(test_target_df, ensemble_weighted)

# 결과 정리
results = {
    'Model': list(predictions.keys()) + ['Ensemble(Mean)', 'Ensemble(Weighted)'],
    'RMSE': [
        np.sqrt(mean_squared_error(test_target_df, predictions[model]))
        for model in predictions.keys()
    ] + [rmse_mean, rmse_weighted],
    'R2': [
        r2_score(test_target_df, predictions[model])
        for model in predictions.keys()
    ] + [r2_mean, r2_weighted],
}

results_df = pd.DataFrame(results)
print("\n=== 모델 성능 비교 ===")
print(results_df.round(4))


/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-01-11 12:23:59,680] A new study created in memory with name: LightGBM


=== 모델 최적화 시작 ===
Optimizing LightGBM...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001316 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1211
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 27
[LightGBM] [Info] Start training from score 62.221487


[I 2025-01-11 12:24:00,767] Trial 0 finished with value: 1.4640699050951034 and parameters: {'n_estimators': 406, 'max_depth': 29, 'learning_rate': 0.029106359131330698, 'num_leaves': 64, 'subsample': 0.6624074561769746, 'colsample_bytree': 0.662397808134481}. Best is trial 0 with value: 1.4640699050951034.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000896 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1211
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 27
[LightGBM] [Info] Start training from score 62.221487


[I 2025-01-11 12:24:01,175] Trial 1 finished with value: 7.068737037916933 and parameters: {'n_estimators': 105, 'max_depth': 27, 'learning_rate': 0.015930522616241012, 'num_leaves': 74, 'subsample': 0.608233797718321, 'colsample_bytree': 0.9879639408647978}. Best is trial 0 with value: 1.4640699050951034.


...

[I 2025-01-11 12:24:46,769] Trial 49 finished with value: 1.4538863914710751 and parameters: {'n_estimators': 388, 'max_depth': 25, 'learning_rate': 0.039682692274679424, 'num_leaves': 36, 'subsample': 0.8217255639612671, 'colsample_bytree': 0.9507013201542869}. Best is trial 20 with value: 1.4078593801347512.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002210 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1211
[LightGBM] [Info] Number of data points in the train set: 5997, number of used features: 27
[LightGBM] [Info] Start training from score 62.221487


/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-01-11 12:24:47,013] A new study created in memory with name: CatBoost


Optimizing CatBoost...


[I 2025-01-11 12:25:15,126] Trial 0 finished with value: 1.6062922264477426 and parameters: {'iterations': 437, 'learning_rate': 0.07969454818643935, 'depth': 12, 'l2_leaf_reg': 0.0024430162614261413, 'bagging_temperature': 0.15601864044243652}. Best is trial 0 with value: 1.6062922264477426.
[I 2025-01-11 12:25:32,203] Trial 1 finished with value: 27.56475323413315 and parameters: {'iterations': 240, 'learning_rate': 0.0013066739238053278, 'depth': 14, 'l2_leaf_reg': 0.002570603566117598, 'bagging_temperature': 0.7080725777960455}. Best is trial 0 with value: 1.6062922264477426.
[I 2025-01-11 12:25:44,565] Trial 2 finished with value: 1.4429982894584386 and parameters: {'iterations': 118, 'learning_rate': 0.08706020878304858, 'depth': 13, 'l2_leaf_reg': 8.148018307012941e-07, 'bagging_temperature': 0.18182496720710062}. Best is trial 2 with value: 1.4429982894584386.
[I 2025-01-11 12:25:47,865] Trial 3 finished with value: 13.67031330834471 and parameters: {'iterations': 265, 'learnin

**Extratree 가 가장 성능이 높게 나타남**

### Extratree Model

In [ ]:
# ExtraTrees 모델의 최적화된 파라미터 가져오기
extra_trees_params = studies['ExtraTrees'].best_params

# ExtraTrees 모델 생성 및 학습
extra_trees_model = ExtraTreesRegressor(**extra_trees_params, random_state=42)
extra_trees_model.fit(train_df, target_df)  # 학습 데이터를 사용하여 모델 학습

# 테스트 데이터 예측
test_predictions = extra_trees_model.predict(test_df)

# RMSE 계산
train_predictions = extra_trees_model.predict(train_df)  # 학습 데이터 예측
train_rmse = np.sqrt(mean_squared_error(target_df, train_predictions))  # 학습 데이터 RMSE
test_rmse = np.sqrt(mean_squared_error(test_target_df, test_predictions))  # 테스트 데이터 RMSE

# 결과 출력
print(f"ExtraTrees 모델 학습 데이터 RMSE: {train_rmse:.4f}")
print(f"ExtraTrees 모델 테스트 데이터 RMSE: {test_rmse:.4f}")

In [ ]:
# 테스트 데이터 예측
test_predictions = extra_trees_model.predict(submit_test_df)

In [ ]:
# sample_submission.csv 불러오기
sample_submission_path = '/content/drive/MyDrive/Colab Notebooks/데이터/sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_path)

# y_pred 값을 sample_submission에 삽입
sample_submission['가격(백만원)'] = test_predictions

# 제출 파일 저장
output_path = '/content/drive/MyDrive/Colab Notebooks/데이터/ET+optuna submission.csv'
sample_submission.to_csv(output_path, index=False)

print(f"Submission file saved to {output_path}")

# 앙상블

In [ ]:
# 각 모델의 예측 수행
weighted_predictions = np.zeros(len(submit_test_df))

# 모델별 가중치 계산
weights = [1 / studies[model].best_value for model in predictions.keys()]
weights = np.array(weights) / sum(weights)  # 가중치 정규화

# 각 모델로 submit_test_df 예측
for i, model_name in enumerate(predictions.keys()):
    # 최적화된 모델 가져오기
    best_model = models[model_name][1](**studies[model_name].best_params, random_state=42)
    best_model.fit(train_df, target_df)  # 모델 학습
    model_predictions = best_model.predict(submit_test_df)  # 테스트 데이터 예측

    # 가중치 적용 후 누적
    weighted_predictions += weights[i] * model_predictions

In [ ]:
# sample_submission.csv 불러오기
sample_submission_path = '/content/drive/MyDrive/Colab Notebooks/데이터/sample_submission.csv'
sample_submission = pd.read_csv(sample_submission_path)

# y_pred 값을 sample_submission에 삽입
sample_submission['가격(백만원)'] = weighted_predictions

# 제출 파일 저장
output_path = '/content/drive/MyDrive/Colab Notebooks/데이터/Ensamble+optuna submission.csv'
sample_submission.to_csv(output_path, index=False)

print(f"Submission file saved to {output_path}")